In [13]:
# Import necessary libraries
import pandas as pd
from gurobipy import Model, GRB

# Load datasets
farms = pd.read_csv('/Users/Sam/Downloads/farms.csv')
processing = pd.read_csv('/Users/Sam/Downloads/processing.csv')
centers = pd.read_csv('/Users/Sam/Downloads/centers.csv')

# Define dimensions
num_farms = len(farms)
num_plants = len(processing)
num_centers = len(centers)

# Extract transportation cost data
transport_cost_farm_to_plant = [[farms.iloc[i, 4+j] for j in range(num_plants)] for i in range(num_farms)]
transport_cost_plant_to_center = [[processing.iloc[j, 4+k] for k in range(num_centers)] for j in range(num_plants)]

# Function to initialize a new model
def initialize_model():
    # Create a new model
    model = Model('BioAgri Optimization')

    # Create decision variables
    x = model.addVars(num_farms, num_plants, vtype=GRB.CONTINUOUS, name="x")  # Raw material transport
    y = model.addVars(num_plants, num_centers, vtype=GRB.CONTINUOUS, name="y")  # Fertilizer transport

    # Objective function: Minimize total cost
    model.setObjective(
        sum(x[i, j] * (farms.loc[i, 'Cost_Per_Ton'] + transport_cost_farm_to_plant[i][j])
            for i in range(num_farms) for j in range(num_plants)) +
        sum(y[j, k] * (processing.loc[j, 'Processing_Cost_Per_Ton'] + transport_cost_plant_to_center[j][k])
            for j in range(num_plants) for k in range(num_centers)),
        GRB.MINIMIZE
    )

    return model, x, y


In [14]:
# Initialize a new model
model, x, y = initialize_model()

# Add basic constraints
for i in range(num_farms):
    model.addConstr(sum(x[i, j] for j in range(num_plants)) <= farms.loc[i, 'Bio_Material_Capacity_Tons'], 
                    name=f"FarmSupply_{i}")
for j in range(num_plants):
    model.addConstr(sum(x[i, j] for i in range(num_farms)) <= processing.loc[j, 'Capacity_Tons'], 
                    name=f"PlantCapacity_{j}")
for k in range(num_centers):
    model.addConstr(sum(y[j, k] for j in range(num_plants)) == centers.loc[k, 'Requested_Demand_Tons'], 
                    name=f"CenterDemand_{k}")
for j in range(num_plants):
    model.addConstr(sum(x[i, j] for i in range(num_farms)) == sum(y[j, k] for k in range(num_centers)), 
                    name=f"FlowBalance_{j}")

# Optimize the model
model.optimize()

# Output results for (a) and (b)
if model.status == GRB.OPTIMAL:
    print("Optimal Cost:", model.objVal)
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 387 rows, 6318 columns and 17118 nonzeros
Model fingerprint: 0x3aa78cc0
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 3e+04]
Presolve time: 0.00s
Presolved: 387 rows, 6318 columns, 17118 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9457439e+05   2.902000e+04   0.000000e+00      0s
     330    2.2970900e+06   0.000000e+00   0.000000e+00      0s

Solved in 330 iterations and 0.01 seconds (0.02 work units)
Optimal objective  2.297089973e+06
Optimal Cost: 2297089.972722625


* c

In [15]:
# Initialize a new model for part (c)
model, x, y = initialize_model()

# Add basic constraints
for i in range(num_farms):
    model.addConstr(sum(x[i, j] for j in range(num_plants)) <= farms.loc[i, 'Bio_Material_Capacity_Tons'], 
                    name=f"FarmSupply_{i}")
for j in range(num_plants):
    model.addConstr(sum(x[i, j] for i in range(num_farms)) <= processing.loc[j, 'Capacity_Tons'], 
                    name=f"PlantCapacity_{j}")
for k in range(num_centers):
    model.addConstr(sum(y[j, k] for j in range(num_plants)) == centers.loc[k, 'Requested_Demand_Tons'], 
                    name=f"CenterDemand_{k}")
for j in range(num_plants):
    model.addConstr(sum(x[i, j] for i in range(num_farms)) == sum(y[j, k] for k in range(num_centers)), 
                    name=f"FlowBalance_{j}")

# Add regional restriction constraints
for j in range(num_plants):
    plant_region = processing.loc[j, 'Region']
    for k in range(num_centers):
        center_region = centers.loc[k, 'Region']
        if plant_region != center_region:
            model.addConstr(y[j, k] == 0, name=f"RegionalRestriction_Plant{j}_Center{k}")

# Optimize the model
model.optimize()

# Output results for (c)
if model.status == GRB.OPTIMAL:
    print("Optimal Cost (Regional Restriction):", model.objVal)
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1761 rows, 6318 columns and 18492 nonzeros
Model fingerprint: 0x9c296277
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 3e+04]
Presolve removed 1374 rows and 1374 columns
Presolve time: 0.00s
Presolved: 387 rows, 4944 columns, 14370 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.2479485e+05   3.627500e+03   0.000000e+00      0s
     320    2.3234885e+06   0.000000e+00   0.000000e+00      0s

Solved in 320 iterations and 0.01 seconds (0.01 work units)
Optimal objective  2.323488497e+06
Optimal Cost (Regional Restriction): 2323488.4970226064


* d

In [16]:
# Initialize a new model for part (d)
model, x, y = initialize_model()

# Add basic constraints
for i in range(num_farms):
    model.addConstr(sum(x[i, j] for j in range(num_plants)) <= farms.loc[i, 'Bio_Material_Capacity_Tons'], 
                    name=f"FarmSupply_{i}")
for j in range(num_plants):
    model.addConstr(sum(x[i, j] for i in range(num_farms)) <= processing.loc[j, 'Capacity_Tons'], 
                    name=f"PlantCapacity_{j}")
for k in range(num_centers):
    model.addConstr(sum(y[j, k] for j in range(num_plants)) == centers.loc[k, 'Requested_Demand_Tons'], 
                    name=f"CenterDemand_{k}")
for j in range(num_plants):
    model.addConstr(sum(x[i, j] for i in range(num_farms)) == sum(y[j, k] for k in range(num_centers)), 
                    name=f"FlowBalance_{j}")

# Add quality restriction constraints
for i in range(num_farms):
    if farms.loc[i, 'Quality'] < 3:
        for j in range(num_plants):
            model.addConstr(x[i, j] == 0, name=f"QualityRestriction_Farm{i}_Plant{j}")

# Optimize the model
model.optimize()

# Output results for (d)
if model.status == GRB.OPTIMAL:
    print("Optimal Cost (Highest Quality Only):", model.objVal)
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3519 rows, 6318 columns and 20250 nonzeros
Model fingerprint: 0xa45bebf3
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 3e+04]
Presolve removed 3306 rows and 3132 columns
Presolve time: 0.00s
Presolved: 213 rows, 3186 columns, 7722 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9457439e+05   3.627500e+03   0.000000e+00      0s
     253    5.7129767e+06   0.000000e+00   0.000000e+00      0s

Solved in 253 iterations and 0.01 seconds (0.01 work units)
Optimal objective  5.712976673e+06
Optimal Cost (Highest Quality Only): 5712976.672909363


* e

In [17]:
# Initialize a new model for part (e) - Sourcing Risk Mitigation
model, x, y = initialize_model()

# Add basic constraints
for i in range(num_farms):
    model.addConstr(sum(x[i, j] for j in range(num_plants)) <= farms.loc[i, 'Bio_Material_Capacity_Tons'], 
                    name=f"FarmSupply_{i}")
for j in range(num_plants):
    model.addConstr(sum(x[i, j] for i in range(num_farms)) <= processing.loc[j, 'Capacity_Tons'], 
                    name=f"PlantCapacity_{j}")
for k in range(num_centers):
    model.addConstr(sum(y[j, k] for j in range(num_plants)) == centers.loc[k, 'Requested_Demand_Tons'], 
                    name=f"CenterDemand_{k}")
for j in range(num_plants):
    model.addConstr(sum(x[i, j] for i in range(num_farms)) == sum(y[j, k] for k in range(num_centers)), 
                    name=f"FlowBalance_{j}")

# Add sourcing risk mitigation constraint
total_supply = farms['Bio_Material_Capacity_Tons'].sum()
for j in range(num_plants):
    model.addConstr(sum(x[i, j] for i in range(num_farms)) <= 0.03 * total_supply, 
                    name=f"SourcingRisk_Plant{j}")

# Optimize the model
model.optimize()

# Output results for (e) Part 1
if model.status == GRB.OPTIMAL:
    print("Optimal Cost (Sourcing Risk Mitigation):", model.objVal)
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 405 rows, 6318 columns and 21600 nonzeros
Model fingerprint: 0x52a9cb0c
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 3e+04]
Presolve removed 18 rows and 0 columns
Presolve time: 0.00s
Presolved: 387 rows, 6318 columns, 17118 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9457439e+05   3.627500e+03   0.000000e+00      0s
     408    2.3117826e+06   0.000000e+00   0.000000e+00      0s

Solved in 408 iterations and 0.01 seconds (0.02 work units)
Optimal objective  2.311782560e+06
Optimal Cost (Sourcing Risk Mitigation): 2311782.559847675


In [18]:
# Initialize a new model for part (e) - Supply Risk Mitigation
model, x, y = initialize_model()

# Add basic constraints
for i in range(num_farms):
    model.addConstr(sum(x[i, j] for j in range(num_plants)) <= farms.loc[i, 'Bio_Material_Capacity_Tons'], 
                    name=f"FarmSupply_{i}")
for j in range(num_plants):
    model.addConstr(sum(x[i, j] for i in range(num_farms)) <= processing.loc[j, 'Capacity_Tons'], 
                    name=f"PlantCapacity_{j}")
for k in range(num_centers):
    model.addConstr(sum(y[j, k] for j in range(num_plants)) == centers.loc[k, 'Requested_Demand_Tons'], 
                    name=f"CenterDemand_{k}")
for j in range(num_plants):
    model.addConstr(sum(x[i, j] for i in range(num_farms)) == sum(y[j, k] for k in range(num_centers)), 
                    name=f"FlowBalance_{j}")

# Add supply risk mitigation constraint
for j in range(num_plants):
    for k in range(num_centers):
        model.addConstr(y[j, k] <= 0.5 * centers.loc[k, 'Requested_Demand_Tons'], 
                        name=f"SupplyRisk_Plant{j}_Center{k}")

# Optimize the model
model.optimize()

# Output results for (e) Part 2
if model.status == GRB.OPTIMAL:
    print("Optimal Cost (Supply Risk Mitigation):", model.objVal)
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2223 rows, 6318 columns and 18954 nonzeros
Model fingerprint: 0x3179cd01
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 3e+04]
Presolve removed 1836 rows and 0 columns
Presolve time: 0.00s
Presolved: 387 rows, 6318 columns, 17118 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9457439e+05   5.441250e+03   0.000000e+00      0s
     398    2.3019498e+06   0.000000e+00   0.000000e+00      0s

Solved in 398 iterations and 0.01 seconds (0.02 work units)
Optimal objective  2.301949785e+06
Optimal Cost (Supply Risk Mitigation): 2301949.784839987


* f

Here’s how to answer question (f) in English, including an analysis of all four scenarios from (c), (d), and the two parts of (e), followed by the combined implementation code.

---

### **Question (f): Explanation and Analysis**

To determine which options are financially defensible, we analyze the results from (c), (d), and (e) in terms of cost impact, operational feasibility, and alignment with strategic goals.

#### **Scenario (c): Regional Restriction**
- **Result**: Optimal cost = **2,323,488.50**.
- **Explanation**: Restricting processing plants to serve home centers only within the same region aligns with practical logistical considerations. While this constraint slightly increases costs compared to a fully flexible model, it reflects realistic limitations, such as transportation efficiency and regulatory boundaries.
- **Conclusion**: **Financially defensible**, as it balances operational feasibility with a manageable cost increase.

#### **Scenario (d): High-Quality Raw Material**
- **Result**: Optimal cost = **5,712,976.67**.
- **Explanation**: Sourcing only high-quality raw materials (quality 3 and 4) nearly doubles the cost compared to (c). While this may improve product quality, the significant cost increase is unlikely to yield proportional benefits in revenue or market competitiveness.
- **Conclusion**: **Not financially defensible**, as the cost increase is excessively high and does not justify the potential benefits.

#### **Scenario (e-1): Sourcing Risk Mitigation**
- **Result**: Optimal cost = **2,311,782.56**.
- **Explanation**: Limiting each facility to processing no more than 3% of the total raw material reduces the risk of over-reliance on specific facilities. The impact on cost is minimal (slightly lower than (c)), demonstrating that this constraint improves risk management without significantly affecting financial performance.
- **Conclusion**: **Financially defensible**, as it enhances supply chain resilience at a negligible cost.

#### **Scenario (e-2): Supply Risk Mitigation**
- **Result**: Optimal cost = **2,301,949.78**.
- **Explanation**: Restricting a processing facility from supplying more than 50% of a single home center’s demand addresses dependency risk. This constraint reduces cost further compared to (c) and (e-1), indicating an improvement in logistical efficiency and risk distribution.
- **Conclusion**: **Financially defensible**, as it reduces supply chain risk while optimizing costs.

---

### **Implementation Plan for All Defensible Options**
Combining all defensible options ((c), (e-1), and (e-2)) offers the best trade-off between cost and risk mitigation. While these constraints introduce operational complexity, they enhance the robustness and sustainability of the supply chain network.

#### **Why This Combination is Financially Defensible**
- Balances cost optimization and risk reduction.
- Reflects realistic logistical and operational limitations.
- Enhances resilience against supply chain disruptions without incurring excessive costs.

---




In [19]:
# Initialize a new model for part (f)
model, x, y = initialize_model()

# Add basic constraints
for i in range(num_farms):
    model.addConstr(sum(x[i, j] for j in range(num_plants)) <= farms.loc[i, 'Bio_Material_Capacity_Tons'], 
                    name=f"FarmSupply_{i}")
for j in range(num_plants):
    model.addConstr(sum(x[i, j] for i in range(num_farms)) <= processing.loc[j, 'Capacity_Tons'], 
                    name=f"PlantCapacity_{j}")
for k in range(num_centers):
    model.addConstr(sum(y[j, k] for j in range(num_plants)) == centers.loc[k, 'Requested_Demand_Tons'], 
                    name=f"CenterDemand_{k}")
for j in range(num_plants):
    model.addConstr(sum(x[i, j] for i in range(num_farms)) == sum(y[j, k] for k in range(num_centers)), 
                    name=f"FlowBalance_{j}")

# Add regional restriction constraints
for j in range(num_plants):
    plant_region = processing.loc[j, 'Region']
    for k in range(num_centers):
        center_region = centers.loc[k, 'Region']
        if plant_region != center_region:
            model.addConstr(y[j, k] == 0, name=f"RegionalRestriction_Plant{j}_Center{k}")
        
# Add sourcing risk mitigation constraint
total_supply = farms['Bio_Material_Capacity_Tons'].sum()
for j in range(num_plants):
    model.addConstr(sum(x[i, j] for i in range(num_farms)) <= 0.03 * total_supply, 
                    name=f"SourcingRisk_Plant{j}")

# Add supply risk mitigation constraint
for j in range(num_plants):
    for k in range(num_centers):
        model.addConstr(y[j, k] <= 0.5 * centers.loc[k, 'Requested_Demand_Tons'], 
                        name=f"SupplyRisk_Plant{j}_Center{k}")

# Optimize the model
model.optimize()

# Output results for (f)
if model.status == GRB.OPTIMAL:
    print("Optimal Cost (All Defensible Options):", model.objVal)
else:
    print("No optimal solution found with all defensible options.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3615 rows, 6318 columns and 24810 nonzeros
Model fingerprint: 0xb36e5572
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 3e+04]
Presolve removed 3228 rows and 1374 columns
Presolve time: 0.00s
Presolved: 387 rows, 4944 columns, 14370 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.2479485e+05   5.441250e+03   0.000000e+00      0s
     397    2.3372934e+06   0.000000e+00   0.000000e+00      0s

Solved in 397 iterations and 0.01 seconds (0.02 work units)
Optimal objective  2.337293375e+06
Optimal Cost (All Defensible Options): 2337293.3745054523


---

### **Expected Outcome**
The result of this combined model will provide the optimal cost when all financially defensible options are implemented together. If needed, the constraints can be further fine-tuned based on business priorities. Let me know if you need additional insights or refinements!

* g

### **Answer for Question (g)**

**Defending the Implementation of All Defensible Options**

Implementing all the defensible options increases the cost compared to the original system, but it represents a strong business decision due to the following key reasons:

1. **Risk Mitigation and Supply Chain Resilience**:
   - **Sourcing Risk Mitigation (e-1)** ensures no single facility becomes a bottleneck, improving the resilience of the supply chain. The cost impact is minimal (Optimal Cost: **2,311,782.56**).
   - **Supply Risk Mitigation (e-2)** prevents over-reliance on specific facilities for fulfilling home center demands, leading to improved distribution (Optimal Cost: **2,301,949.78**).

2. **Operational Feasibility**:
   - **Regional Restrictions (c)** reflect logistical realities, ensuring that processing plants serve nearby home centers, which aligns with real-world transportation limitations (Optimal Cost: **2,323,488.50**).

3. **Strategic Cost Control**:
   - The combined defensible options (f) result in an optimal cost of **2,337,293.37**, which is only a 0.6% increase over (c). This small cost increment is justified by the enhanced robustness and risk mitigation achieved.

4. **Avoiding Unsustainable Cost Increases**:
   - The high-quality raw material constraint (d) results in a significant cost increase to **5,712,976.67** (+145% vs. (c)), which is financially indefensible compared to the combined defensible options.

**Conclusion**:
While the combined defensible options increase costs slightly, they significantly enhance the robustness, feasibility, and sustainability of the supply chain. This approach ensures long-term resilience and operational efficiency, justifying the additional cost.

* h

In [20]:
# Function to test feasibility at different sourcing risk mitigation percentages
def test_sourcing_risk_threshold(starting_percentage, step_size):
    percentage = starting_percentage
    while percentage > 0:
        # Initialize a new model for each percentage
        model, x, y = initialize_model()

        # Add basic constraints
        for i in range(num_farms):
            model.addConstr(sum(x[i, j] for j in range(num_plants)) <= farms.loc[i, 'Bio_Material_Capacity_Tons'], 
                            name=f"FarmSupply_{i}")
        for j in range(num_plants):
            model.addConstr(sum(x[i, j] for i in range(num_farms)) <= processing.loc[j, 'Capacity_Tons'], 
                            name=f"PlantCapacity_{j}")
        for k in range(num_centers):
            model.addConstr(sum(y[j, k] for j in range(num_plants)) == centers.loc[k, 'Requested_Demand_Tons'], 
                            name=f"CenterDemand_{k}")
        for j in range(num_plants):
            model.addConstr(sum(x[i, j] for i in range(num_farms)) == sum(y[j, k] for k in range(num_centers)), 
                            name=f"FlowBalance_{j}")

        # Add regional restriction constraints
        for j in range(num_plants):
            plant_region = processing.loc[j, 'Region']
            for k in range(num_centers):
                center_region = centers.loc[k, 'Region']
                if plant_region != center_region:
                    model.addConstr(y[j, k] == 0, name=f"RegionalRestriction_Plant{j}_Center{k}")

        # Add sourcing risk mitigation constraint with the current percentage
        total_supply = farms['Bio_Material_Capacity_Tons'].sum()
        for j in range(num_plants):
            model.addConstr(sum(x[i, j] for i in range(num_farms)) <= (percentage / 100) * total_supply, 
                            name=f"SourcingRisk_Plant{j}")

        # Add supply risk mitigation constraint
        for j in range(num_plants):
            for k in range(num_centers):
                model.addConstr(y[j, k] <= 0.5 * centers.loc[k, 'Requested_Demand_Tons'], 
                                name=f"SupplyRisk_Plant{j}_Center{k}")

        # Optimize the model
        model.optimize()

        # Check feasibility
        if model.status == GRB.INFEASIBLE:
            print(f"Infeasibility reached at sourcing risk mitigation percentage: {percentage:.1f}%")
            return percentage
        percentage -= step_size  # Reduce percentage by step size

    print("Model remained feasible for all tested percentages.")
    return 0  # If no infeasibility is found


In [21]:
# Test the threshold for sourcing risk mitigation
threshold = test_sourcing_risk_threshold(starting_percentage=3.0, step_size=0.1)
print(f"Threshold for sourcing risk mitigation: {threshold:.1f}%")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3615 rows, 6318 columns and 24810 nonzeros
Model fingerprint: 0xb36e5572
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 3e+04]
Presolve removed 3228 rows and 1374 columns
Presolve time: 0.01s
Presolved: 387 rows, 4944 columns, 14370 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.2479485e+05   5.441250e+03   0.000000e+00      0s
     397    2.3372934e+06   0.000000e+00   0.000000e+00      0s

Solved in 397 iterations and 0.01 seconds (0.02 work units)
Optimal objective  2.337293375e+06
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors

### **Answer for Question (h)**

#### **Key Findings**
1. **Threshold for Feasibility**:
   - The supply chain becomes infeasible when the sourcing risk mitigation limit drops below **2.4%**.
   - At **2.5%**, the model remains feasible with an optimal transportation and procurement cost of **$5,755,911.11**.

2. **Managerial Interpretation**:
   - **Capacity Limits**: The result highlights that the supply chain has limited flexibility to absorb tighter restrictions on sourcing. Dropping the limit below **2.5%** creates excessive reliance on a few facilities, making the network unsustainable.
   - **Risk-Resilience Trade-Off**: While reducing the sourcing risk mitigation percentage might seem desirable to lower dependency risks, the supply chain cannot operate below the threshold due to resource and capacity constraints.

3. **Implications for Supply Chain Management**:
   - **Balance Constraints**: Managers must carefully balance risk mitigation policies with operational feasibility to avoid disruption.
   - **Expand Capacity**: To implement stricter sourcing limits, additional investments in facility capacity or diversification of suppliers may be necessary.
   - **Monitor Critical Limits**: The **2.5% threshold** serves as a guide for future policy development, ensuring the supply chain remains robust while addressing risk.

4. **Conclusion**:
   - The supply chain’s limited capacity for risk mitigation emphasizes the need for strategic adjustments rather than imposing overly restrictive policies. Maintaining feasible limits is crucial for sustaining operations and achieving long-term goals.

* Question B

In [27]:
import pandas as pd
from gurobipy import Model, GRB

# Load the dataset
gym = pd.read_csv('/Users/Sam/Downloads/updated_gym_data.csv')

# Initialize the Gurobi model
model = Model("Weightlifting_Program")

# Number of exercises
n_exercises = gym.shape[0]

# Decision variables: Proportion of workout allocated to each exercise
# Variable names are generated dynamically based on exercise indices
x = model.addVars(n_exercises, lb=0, ub=1, name="x")

# Total allocation constraint: Sum of proportions must equal 1
model.addConstr(x.sum() == 1, "TotalAllocation")

# Print the number of decision variables
print(f"Number of decision variables: {n_exercises}")


Number of decision variables: 2637


# Decision Variables

Each exercise in the dataset represents a potential decision variable. Since the dataset contains 2,637 exercises, there are 2,637 decision variables. Each variable \(x_i\) corresponds to the proportion of the workout program allocated to exercise \(i\).


In [26]:
gym.head()

,Exercise,Category,BodyPart,Equipment,Difficulty,Stimulus-to-Fatigue,Expected Time,Hypertrophy Rating
0,Bench Press With Short Bands,Powerlifting,Chest,Bands,Beginner,0.817884,15.518089,0.596124
1,Hip Lift with Band,Powerlifting,Glutes,Bands,Beginner,0.768902,14.655351,0.623237
2,Band Good Morning (Pull Through),Powerlifting,Hamstrings,Bands,Beginner,0.792188,16.292358,0.601159
3,Speed Box Squat,Powerlifting,Quadriceps,Bands,Intermediate,0.599044,17.109781,0.800347
4,Partner plank band row,Strength,Abdominals,Bands,Intermediate,0.730726,14.212727,0.461565
